**Βήμα 11α: Transfer Learning**
 Για την υλοποίηση transfer learning η βασική ιδέα είναι ότι εκπαιδεύουμε ένα μοντέλο σε dataset το οποίο έχει μεγαλύτερο μέγεθος ώστε να εκπαιδευτεί το μοντέλο καλύτερα στο γενικότερο εύρος της πληροφορίας (τα dataset πρεπει να είναι παρόμοιο περιεχομένου). Στην συνέχεια, μετά την γενική εκπαίδευση του μοντέλου (στην οποία κρατάμε το καλύτερο με χρήση checkpoints) αφαιρούμε τα τελευταία layers τα οποία εμπεριέχουν την ειδική πληροφορία και επανεκπαιδεύουμε το μοντέλο στο δικό μας dataset (για λιγότερες εποχές) κρατώντας ίδια τα βάρη των layers που αφήσαμε και προσθέτοντας στην θέση των τελευταίων που αφαιεσαμε άλλα τα οποία αρχικοποιούνται τυχαία.
 Έτσι μαθαίνουμε τα τελευταία layers στην ειδική πλροφορία του dataset μας.
 
 Παρατηρούμε ότι το transfer learning που εφαρμόσαμε δεν είχε τόσο μεγάλη επιτυχία,καθώς δεν ήταν τόσο καλά τα αποτελέσματα οσο το βήμα 10.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['data']


In [2]:
import numpy as np
import gzip
import copy
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset
from torch.utils.data import SubsetRandomSampler, DataLoader
import os

class_mapping = {
    'Rock': 'Rock',
    'Psych-Rock': 'Rock',
    'Indie-Rock': None,
    'Post-Rock': 'Rock',
    'Psych-Folk': 'Folk',
    'Folk': 'Folk',
    'Metal': 'Metal',
    'Punk': 'Metal',
    'Post-Punk': None,
    'Trip-Hop': 'Trip-Hop',
    'Pop': 'Pop',
    'Electronic': 'Electronic',
    'Hip-Hop': 'Hip-Hop',
    'Classical': 'Classical',
    'Blues': 'Blues',
    'Chiptune': 'Electronic',
    'Jazz': 'Jazz',
    'Soundtrack': None,
    'International': None,
    'Old-Time': None
}


def torch_train_val_split(
        dataset, batch_train, batch_eval,
        val_size=.2, shuffle=True, seed=42):
    # Creating data indices for training and validation splits:
    dataset_size = len(dataset)
    indices = list(range(dataset_size))
    val_split = int(np.floor(val_size * dataset_size))
    if shuffle:
        np.random.seed(seed)
        np.random.shuffle(indices)
    train_indices = indices[val_split:]
    val_indices = indices[:val_split]

    # Creating PT data samplers and loaders:
    train_sampler = SubsetRandomSampler(train_indices)
    val_sampler = SubsetRandomSampler(val_indices)

    train_loader = DataLoader(dataset,
                              batch_size=batch_train,
                              sampler=train_sampler)
    val_loader = DataLoader(dataset,
                            batch_size=batch_eval,
                            sampler=val_sampler)
    return train_loader, val_loader


def read_spectrogram(spectrogram_file, chroma=True):
    with gzip.GzipFile(spectrogram_file, 'r') as f:
        spectrograms = np.load(f)
    # spectrograms contains a fused mel spectrogram and chromagram
    # Decompose as follows
    return spectrograms.T


class LabelTransformer(LabelEncoder):
    def inverse(self, y):
        try:
            return super(LabelTransformer, self).inverse_transform(y)
        except:
            return super(LabelTransformer, self).inverse_transform([y])

    def transform(self, y):
        try:
            return super(LabelTransformer, self).transform(y)
        except:
            return super(LabelTransformer, self).transform([y])

        
class PaddingTransform(object):
    def __init__(self, max_length, padding_value=0):
        self.max_length = max_length
        self.padding_value = padding_value

    def __call__(self, s):
        if len(s) == self.max_length:
            return s

        if len(s) > self.max_length:
            return s[:self.max_length]

        if len(s) < self.max_length:
            s1 = copy.deepcopy(s)
            pad = np.zeros((self.max_length - s.shape[0], s.shape[1]), dtype=np.float32)
            s1 = np.vstack((s1, pad))
            return s1

        
class SpectrogramDataset(Dataset):
    def __init__(self, path, class_mapping=None, train=True, max_length=-1):
        t = 'train' if train else 'test'
        p = os.path.join(path, t)
        self.index = os.path.join(path, "{}_labels.txt".format(t))
        #print(self.index)
        self.files, labels = self.get_files_labels(self.index, class_mapping)
        self.feats = [read_spectrogram(os.path.join(p, f)) for f in self.files]
        self.feat_dim = self.feats[0].shape[1]
        self.lengths = [len(i) for i in self.feats]
        self.max_length = max(self.lengths) if max_length <= 0 else max_length
        self.zero_pad_and_stack = PaddingTransform(self.max_length)
        self.label_transformer = LabelTransformer()
        if isinstance(labels, (list, tuple)):
            self.labels = np.array(self.label_transformer.fit_transform(labels)).astype('int64')

    def get_files_labels(self, txt, class_mapping):
        with open(txt, 'r') as fd:
            lines = [l.rstrip().split('\t') for l in fd.readlines()[1:]]
        files, labels = [], []
        for l in lines:
            label = l[1]
            if class_mapping:
                label = class_mapping[l[1]]
            if not label:
                continue
            files.append(l[0])
            labels.append(label)
        return files, labels

    def __getitem__(self, item):
        l = min(self.lengths[item], self.max_length)
        return self.zero_pad_and_stack(self.feats[item]), self.labels[item], l

    def __len__(self):
        return len(self.labels)

In [3]:
BATCH_SZ=32

specs = SpectrogramDataset('../input/data/data/fma_genre_spectrograms/', train=True, class_mapping=class_mapping, max_length=-1)
train_loader, val_loader = torch_train_val_split(specs, BATCH_SZ ,BATCH_SZ, val_size=.33)
test_loader = DataLoader(SpectrogramDataset('../input/data/data/fma_genre_spectrograms/', train=False, class_mapping=class_mapping, max_length=-1))

In [4]:
import numpy as np
import torch
from torch.utils.data import Dataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class ConvNet(nn.Module):
    def __init__(self,input_channels, num_classes):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(input_channels, 4, kernel_size=(3,3), stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(4),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(4, 16, kernel_size=(3,3), stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(16 , 32 , kernel_size=(3,3), stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(kernel_size=3, stride=3)
        )
        self.layer4 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=(3,3), stride=1, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size=3, stride=3)
        )
        
        self.dense1= nn.Linear(6720,500) 
        self.dense2 = nn.Linear(500,10)
        
        
    def forward(self, x):
        #print(x.shape)
        x = x.transpose(1, 2)
        #print(x.shape)
        x.unsqueeze_(1)
        #print(x.shape)
        out1 = self.layer1(x)
        #print(out1.shape)
        out2= self.layer2(out1)
        #print(out2.shape)
        out3= self.layer3(out2)
        #print(out3.shape)
        out4= self.layer4(out3)
        #print(out4.shape)
        
    
        out_flat=out4.reshape(-1,out4.size(1)*out4.size(2)*out4.size(3))
        #print(out_flat.shape)
        
        
        #implementing fully connected layers
        
        hidden_out = self.dense1(out_flat)
        final_out = self.dense2(hidden_out)
        
        return final_out

In [5]:


class Trainer_with_Checkpoints():
    def __init__(self,validate_every,metrics,max_epochs,patience=10):
    
        self.validate_every=validate_every
        self.metrics = metrics
        self.patience=patience
        self.best_score=None
        self.max_epochs = max_epochs
        
        
    def validate_accuracy(self,mymodel,validation_batches):
        with torch.no_grad():
            mymodel.eval()
            num_correct=0
            num_samples=0
            with torch.no_grad():
                for index, instance in enumerate(validation_batches):
                    features = instance[:][0].to(device)
                    labels = instance[:][1].to(device)
                    lengths = instance[:][2].to(device)
                    features = features.type(torch.FloatTensor).to(device)
                    
                    out = mymodel(features)
                    out_scores = F.log_softmax(out,dim=1)
                    value, y_pred = out_scores.max(1)

                    num_correct += (labels == y_pred).sum().detach().item()
                    num_samples += features.shape[0]

                print("Score for validation set: " ,num_correct / num_samples)
        return num_correct/num_samples

    
    def checkpoint(self,mymodel,myoptimizer,epoch,checkpointdir,myscheduler=None):
        
        #if myscheduler is not None:
         #   state = {'epoch': epoch + 1,'state_dict': mymodel.state_dict(),
       #              'optim_dict' : myoptimizer.state_dict(),'scheduler_dict' : myscheduler.state_dict()}
        #else:
        #    state = {'epoch': epoch + 1,'state_dict': mymodel.state_dict(),'optim_dict' : myoptimizer.state_dict()}
        
        #utils.save_checkpoint(state,checkpoint=self.checkpointdir) # path to folder
        torch.save({
            'epoch': epoch,
            'model_state_dict': mymodel.state_dict(),
            'optimizer_state_dict': myoptimizer.state_dict(),
            }, checkpointdir)
        return
    
    
    def train_model(self,mymodel,myoptimizer,myloss_function,training_batches,validation_batches,
                    checkpointdir,myscheduler=None):
        
        self.best_score=None
        counter =0
        device=torch.device("cuda")
        if self.patience < 1:
            raise ValueError("Argument patience should be positive integer")
        
        
        
        for epoch in range(self.max_epochs):
            #no need to set requires_grad=True for parameters(weights) as it done by default. Also for input requires_grad is not
            #always necessary. So we comment the following line.
            #with torch.autograd(): 
            mymodel.train()

            if myscheduler is not None:
                myscheduler.step()

            running_average_loss = 0


            #train model in each epoch
            for index,instance in enumerate(training_batches):
                # Step 1. Remember that Pytorch accumulates gradients.
                # We need to clear them out before each instance
                features = instance[:][0].to(device)
                labels = instance[:][1].to(device)
                lengths = instance[:][2].to(device)
                features = features.type(torch.FloatTensor).to(device)
                
                myoptimizer.zero_grad()
                
                prediction_vec = mymodel(features)
                prediction_vec.to(device)
                
                myloss = myloss_function(prediction_vec,labels)
                myloss.backward(retain_graph=True)
                myoptimizer.step()
                running_average_loss += myloss.detach().item()
                if index % 100 == 0:
                    print("Epoch: {} \t Batch: {} \t Training Loss {}".format(epoch, index, float(running_average_loss) / (index + 1)))
               
            if epoch==self.max_epochs-1:
                print("yyyyyeaaaaahhhh")
                if 'accuracy' in self.metrics:
                    score = self.validate_accuracy(mymodel,validation_batches)

                if self.best_score is None:
                    self.best_score = score
                    self.checkpoint(mymodel,myoptimizer,epoch,checkpointdir,myscheduler)
                    print("checkpoint done!")
                    
                elif score < self.best_score:
                    counter += 1
                    if counter >= self.patience:
                        print("EarlyStopping: Stop training")
                        return
                else:
                    #found better state in our model
                    self.best_score = score
                    counter = 0
                    #checkpoint
                    self.checkpoint(mymodel,myoptimizer,epoch,checkpointdir,myscheduler)
                    print("checkpoint done!")
            
            if epoch % self.validate_every == 0:
                if 'accuracy' in self.metrics:
                    score = self.validate_accuracy(mymodel,validation_batches)

                if self.best_score is None:
                    self.best_score = score
                    #checkpoint
                    self.checkpoint(mymodel,myoptimizer,epoch,checkpointdir,myscheduler)
                    print("checkpoint done!")
                    
                elif score < self.best_score:
                    counter += 1
                    if counter >= self.patience:
                        print("EarlyStopping: Stop training")
                        return
                    
                else:
                    #found better state in our model
                    self.best_score = score
                    counter = 0
                    #checkpoint
                    self.checkpoint(mymodel,myoptimizer,epoch,checkpointdir,myscheduler)
                    print("checkpoint done!")

In [6]:
VALIDATE_EVERY=5
METRICS='accuracy'
MAX_EPOCHS=40
PATIENCE=3



CHECKDIR='./model_tranfer.pt'

input_channels=1
num_classes=10
device=torch.device("cuda")
model3 = ConvNet(input_channels,num_classes)
model3.to(device)





# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model3.parameters(),lr=0.01)


trainer = Trainer_with_Checkpoints(validate_every=VALIDATE_EVERY,metrics=METRICS,max_epochs=MAX_EPOCHS,patience=PATIENCE)

trainer.train_model(mymodel=model3,myoptimizer=optimizer,myloss_function=criterion,training_batches=train_loader,
                    validation_batches=val_loader,checkpointdir=CHECKDIR)

Epoch: 0 	 Batch: 0 	 Training Loss 2.3606491088867188
Score for validation set:  0.23684210526315788
checkpoint done!
Epoch: 1 	 Batch: 0 	 Training Loss 2.246367931365967
Epoch: 2 	 Batch: 0 	 Training Loss 2.0726478099823
Epoch: 3 	 Batch: 0 	 Training Loss 1.9512250423431396
Epoch: 4 	 Batch: 0 	 Training Loss 1.7854173183441162
Epoch: 5 	 Batch: 0 	 Training Loss 2.1310973167419434
Score for validation set:  0.12763157894736843
Epoch: 6 	 Batch: 0 	 Training Loss 1.5493055582046509
Epoch: 7 	 Batch: 0 	 Training Loss 2.8074753284454346
Epoch: 8 	 Batch: 0 	 Training Loss 1.6358164548873901
Epoch: 9 	 Batch: 0 	 Training Loss 1.408928394317627
Epoch: 10 	 Batch: 0 	 Training Loss 0.538772702217102
Score for validation set:  0.13026315789473683
Epoch: 11 	 Batch: 0 	 Training Loss 1.1837912797927856
Epoch: 12 	 Batch: 0 	 Training Loss 0.4012756943702698
Epoch: 13 	 Batch: 0 	 Training Loss 0.8012294173240662
Epoch: 14 	 Batch: 0 	 Training Loss 1.2533838748931885
Epoch: 15 	 Batch:

In [7]:
import numpy as np
import gzip
import copy
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset
from torch.utils.data import SubsetRandomSampler, DataLoader
import os


class_mapping = {
    'Rock': 'Rock',
    'Psych-Rock': 'Rock',
    'Indie-Rock': None,
    'Post-Rock': 'Rock',
    'Psych-Folk': 'Folk',
    'Folk': 'Folk',
    'Metal': 'Metal',
    'Punk': 'Metal',
    'Post-Punk': None,
    'Trip-Hop': 'Trip-Hop',
    'Pop': 'Pop',
    'Electronic': 'Electronic',
    'Hip-Hop': 'Hip-Hop',
    'Classical': 'Classical',
    'Blues': 'Blues',
    'Chiptune': 'Electronic',
    'Jazz': 'Jazz',
    'Soundtrack': None,
    'International': None,
    'Old-Time': None
}


def torch_train_val_split(
        dataset, batch_train, batch_eval,
        val_size=.2, shuffle=True, seed=42):
    # Creating data indices for training and validation splits:
    dataset_size = len(dataset)
    indices = list(range(dataset_size))
    val_split = int(np.floor(val_size * dataset_size))
    if shuffle:
        np.random.seed(seed)
        np.random.shuffle(indices)
    train_indices = indices[val_split:]
    val_indices = indices[:val_split]

    # Creating PT data samplers and loaders:
    train_sampler = SubsetRandomSampler(train_indices)
    val_sampler = SubsetRandomSampler(val_indices)

    train_loader = DataLoader(dataset,
                              batch_size=batch_train,
                              sampler=train_sampler)
    val_loader = DataLoader(dataset,
                            batch_size=batch_eval,
                            sampler=val_sampler)
    return train_loader, val_loader


def read_spectrogram(spectrogram_file, chroma=True):
    with gzip.GzipFile(spectrogram_file, 'r') as f:
        spectrograms = np.load(f)
    # spectrograms contains a fused mel spectrogram and chromagram
    # Decompose as follows
    return spectrograms.T


class LabelTransformer(LabelEncoder):
    def inverse(self, y):
        try:
            return super(LabelTransformer, self).inverse_transform(y)
        except:
            return super(LabelTransformer, self).inverse_transform([y])

    def transform(self, y):
        try:
            return super(LabelTransformer, self).transform(y)
        except:
            return super(LabelTransformer, self).transform([y])

        
class PaddingTransform(object):
    def __init__(self, max_length, padding_value=0):
        self.max_length = max_length
        self.padding_value = padding_value

    def __call__(self, s):
        if len(s) == self.max_length:
            return s

        if len(s) > self.max_length:
            return s[:self.max_length]

        if len(s) < self.max_length:
            s1 = copy.deepcopy(s)
            pad = np.zeros((self.max_length - s.shape[0], s.shape[1]), dtype=np.float32)
            s1 = np.vstack((s1, pad))
            return s1

        
class SpectrogramDataset(Dataset):
    def __init__(self, path, class_mapping=None, train=True, max_length=-1):
        t = 'train' if train else 'test'
        p = os.path.join(path, t)
        self.index = os.path.join(path, "{}_labels.txt".format(t))
        self.files, labels = self.get_files_labels(self.index, class_mapping)
        #print(self.files)
        
        self.feats = [read_spectrogram(os.path.join(p, f+".fused.full.npy.gz")) for f in self.files]
        self.feat_dim = self.feats[0].shape[1]
        self.lengths = [len(i) for i in self.feats]
        self.max_length = max(self.lengths) if max_length <= 0 else max_length
        self.zero_pad_and_stack = PaddingTransform(self.max_length)
        #self.label_transformer = LabelTransformer()
        #if isinstance(labels, (list, tuple)):
            #self.labels = np.array(self.label_transformer.fit_transform(labels)).astype('int64')
        self.labels=labels
    def get_files_labels(self, txt, class_mapping):
        with open(txt, 'r') as fd:
            lines = [l.rstrip().split('\t') for l in fd.readlines()[1:]]
            
        files, labels = [], []
        for l in lines:
            l=l[0].split(",")
            b=l[1:]
            b = list(map(float,b))
            files.append(l[0])
            
            labels.append(b)
        return files, labels

    def __getitem__(self, item):
        l = min(self.lengths[item], self.max_length)
        return self.zero_pad_and_stack(self.feats[item]), self.labels[item], l

    def __len__(self):
        return len(self.labels)

In [8]:


BATCH_SZ=32

specs = SpectrogramDataset('../input/data/data/multitask_dataset/', train=True, class_mapping=class_mapping, max_length=-1)
train_loader, val_loader = torch_train_val_split(specs, BATCH_SZ ,BATCH_SZ, val_size=.33)



In [9]:
checkpoint = torch.load(CHECKDIR)

model3.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

In [10]:
for param in model3.parameters():
    param.requires_grad=False
    
model3.dense1= nn.Linear(6720,500) 
model3.dense2 = nn.Linear(500,50)
model3.dense3 = nn.Linear(50,1)
model3.to(device)

ConvNet(
  (layer1): Sequential(
    (0): Conv2d(1, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(4, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  )
  (layer4): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    

In [11]:
# Loss and optimizer

num_epochs=10
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model3.parameters())



for epoch in range(num_epochs):
    #no need to set requires_grad=True for parameters(weights) as it done by default. Also for input requires_grad is not
    #always necessary. So we comment the following line.
    #with torch.autograd(): 
    model3.train()
    #scheduler.step()
    running_average_loss = 0

    #train model in each epoch
    for index,instance in enumerate(train_loader):
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        #features,labels,lengths=instance
        
        features = instance[:][0].to(device)
        labels = instance[:][1]
        valence_labels = labels[0].type(torch.FloatTensor).to(device)
        energy_labels = labels[1].type(torch.FloatTensor).to(device)
        dance_labels = labels[2].type(torch.FloatTensor).to(device)
        lengths = instance[:][2].to(device)
        features = features.type(torch.FloatTensor).to(device)

        optimizer.zero_grad()
        
        # Step 3. Run our forward pass.
        prediction_vec = model3(features)
        prediction_vec.to(device)
        
        
        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        energy_labels = energy_labels.unsqueeze(1)
        
        loss = criterion(prediction_vec,energy_labels)
        loss.backward(retain_graph=True)
        optimizer.step()

        running_average_loss += loss.detach().item()
    print("Epoch: {} \t \t Training Loss {}".format(epoch, float(running_average_loss) / (index + 1)))

Epoch: 0 	 	 Training Loss 2.687690374751886
Epoch: 1 	 	 Training Loss 1.189791811009248
Epoch: 2 	 	 Training Loss 1.2360031033555667
Epoch: 3 	 	 Training Loss 1.3810958390434582
Epoch: 4 	 	 Training Loss 1.0928535958131154
Epoch: 5 	 	 Training Loss 0.8533485345542431
Epoch: 6 	 	 Training Loss 0.6935931220650673
Epoch: 7 	 	 Training Loss 0.4840547790129979
Epoch: 8 	 	 Training Loss 0.3497561203936736
Epoch: 9 	 	 Training Loss 0.2464602943509817


In [12]:
from scipy import stats
model3.eval()

n_samples = 0
SE = 0
spearman=[]
running_average_loss=0
with torch.no_grad():
    for index, instance in enumerate(val_loader):
        features = instance[:][0].to(device)
        labels = instance[:][1]
        valence_labels = labels[0].type(torch.FloatTensor).to(device)
        energy_labels = labels[1].type(torch.FloatTensor).to(device)
        dance_labels = labels[2].type(torch.FloatTensor).to(device)
        lengths = instance[:][2].to(device)
        features = features.type(torch.FloatTensor).to(device)
        
        
    
        out = model3(features)
        out = out.to(device)
        #print(out)
        #print(valence_labels)

        energy_labels = energy_labels.unsqueeze(1)

        
        spearman.append(stats.spearmanr(energy_labels.cpu().squeeze(),out.cpu().squeeze(),axis=0)[0])

print("Spearnman's correlation for CNN-2d in validation set (predicting energy): " , np.mean(spearman) )

Spearnman's correlation for CNN-2d in validation set (predicting energy):  0.2647716075704687
